%pip install numpy==1.26.2

In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import logging

debug = logging.getLogger("Debug")
info  = print
plt.ion()   # interactive mode

In [2]:
#check GPU
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running CUDA Mode:", device, torch.cuda.get_device_name(0))
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Running MPS Mode:", device)
else:
    device = torch.device("cpu")
    print("Running CPU Mode:", device)



Running CUDA Mode: cuda NVIDIA A100-PCIE-40GB


## Data and Classes
- Create Dataloader class

Note: Working on Part (a) as of now.  
Guiding light: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [3]:
START_TOKEN = "START"
END_TOKEN = "END"
UNK_TOKEN = "UNK"

# MAX_EXAMPLES = 100
class Vocabulary:
    def __init__(self, freq_dict, wd_to_id, id_to_wd):
        self.freq_dict = freq_dict
        self.wd_to_id = wd_to_id
        self.id_to_wd = id_to_wd
        self.N = len(freq_dict)
    
    def get_id(self, word):
        if word in self.wd_to_id:
            return self.wd_to_id[word]
        else:
            return self.wd_to_id[UNK_TOKEN]

class LatexFormulaDataset(Dataset):
    """Latex Formula Dataset: Image and Text"""
    
    def __init__(self, csv_file, root_dir, max_examples=None, transform = None):
        """
        Arguments:
            csv_file (string): Path to the csv file with image name and text
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #@TODO: May want to preload images
        info("Loading Dataset...")
        self.df = pd.read_csv(csv_file)
        info("Loaded Dataset", self.df.info)
        
        #Slice the dataset if max_examples is not None
        if max_examples is not None:
            self.df = self.df.iloc[:max_examples, :]

        self.root_dir = root_dir
        self.transform = transform
        self.vocab= self.construct_vocab()  

        self.maxlen = 0
        for formula in self.df['formula']:
            if len(formula) > self.maxlen:
                self.maxlen = len(formula)

        self.df['formula'] = self.df['formula'].apply(lambda x: [START_TOKEN] + x + [END_TOKEN] + [UNK_TOKEN]*(self.maxlen - len(x)))
        self.maxlen += 2 #for start and end tokens

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        """
        Returns sample of type image, textformula
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.df.iloc[idx, 0])
        image = io.imread(img_name)
        formula = self.df.iloc[idx, 1]
        formula = np.array([formula], dtype=str).reshape(-1, 1)
        formula = [[self.vocab.get_id(wd[0]) for wd in formula]] 
        sample = {'image': image, 'formula': torch.tensor(formula, dtype=torch.int64)}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
            
        return sample 
    
    def construct_vocab(self):
        """
        Constructs vocabulary from the dataset formulas
        """
        #Split on spaces to tokenize
        self.df['formula'] = self.df['formula'].apply(lambda x: x.split())

        freq_dict = {}
        for formula in self.df['formula']:
            for wd in formula:
                if wd not in freq_dict:
                    freq_dict[wd] = 1
                else:
                    freq_dict[wd] += 1
        freq_dict[START_TOKEN] = 1
        freq_dict[END_TOKEN] = 1
        freq_dict[UNK_TOKEN] = 1
        N = len(freq_dict)
        wd_to_id = {}
        for i, wd in enumerate(freq_dict):
            wd_to_id[wd] = i
        id_to_wd = {v: k for k, v in wd_to_id.items()}
    
        #pad the formulas with 
        return Vocabulary(freq_dict, wd_to_id, id_to_wd)      

def get_dataloader(csv_path, image_root, batch_size, transform = None, max_examples = None):
    """
    Returns dataloader,dataset for the dataset
    """
    dataset = LatexFormulaDataset(csv_path, image_root, max_examples=max_examples,transform=transform) #checked
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader, dataset
     

### Encoder Network
- A CNN to encode image to more meaningful vector

In [4]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super().__init__()
    
        #@TODO:reduce number of layers: eliminate pools and acts
        self.conv1 = nn.Conv2d(3, 32, (5, 5))
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d((2, 2))
        
        self.conv2 = nn.Conv2d(32, 64, (5, 5))
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d((2, 2))
        
        self.conv3 = nn.Conv2d(64, 128, (5, 5))
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d((2, 2))
        
        self.conv4 = nn.Conv2d(128, 256, (5, 5))
        self.act4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d((2, 2))
        
        self.conv5 = nn.Conv2d(256, 512, (5, 5))
        self.act5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d((2, 2))
        
        self.avg_pool = nn.AvgPool2d((3, 3))
    
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.pool1(x)
        
        x = self.act2(self.conv2(x))
        x = self.pool2(x)
        
        x = self.act3(self.conv3(x))
        x = self.pool3(x)
        
        x = self.act4(self.conv4(x))
        x = self.pool4(x)
        
        x = self.act5(self.conv5(x))
        x = self.pool5(x)
        
        x = self.avg_pool(x)
        x = x.view(-1,512) 
        # info(f"Encoder Output Shape: {x.shape}")
        return x

### Vocabulary
- https://github.com/harvardnlp/im2markup/blob/master

### Decoder Network

In [5]:
class Decoder(nn.Module):
    """
    Inputs:
    (here M is whatever the batch size is passed)

    context_size : size of the context vector [shape: (1,M,context_size)]
    n_layers: number of layers [for our purposes, defaults to 1]
    hidden_size : size of the hidden state vectors [shape: (n_layers,M,hidden_size)]
    embed_size : size of the embedding vectors [shape: (1,M,embed_size)]
    vocab_size : size of the vocabulary
    max_length : maximum length of the formula
    """
    def __init__(self, context_size, vocab, n_layers = 1, hidden_size = 512, embed_size = 512,  max_length = 100):
        super().__init__()
        self.context_size = context_size
        self.vocab = vocab
        self.vocab_size = vocab.N
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.max_length = max_length


        self.input_size = context_size + embed_size

        self.embed = nn.Embedding(self.vocab_size, embed_size)
        self.lstm = nn.LSTMCell(self.input_size, self.hidden_size)
        self.linear = nn.Linear(hidden_size, self.vocab_size)
        self.softmax = nn.Softmax(dim = 1)
    
    def forward(self, context, target_tensor = None):
        """
        M: batch_size
        context is the context vector from the encoder [shape: (M,context_size)]
        target_tensor is the formula in tensor form [shape: (M,max_length)] (in the second dimension, it is sequence of indices of formula tokens)
            if target_tensor is not None, then we are in Teacher Forcing mode
            else normal jo bhi (last prediction ka embed is concatenated)
        """
        # info("Decoder Forward")
        # info(f"Context shape: {context.shape}")
        context.to(device)
        target_tensor = target_tensor.squeeze()
        batch_size = context.shape[0]

        #initialize hidden state and cell state
            #@TODO: Some caveat in the size of the cell state. Should it be same as hidden_size? (check nn.LSTM documentation)
        hidden = torch.zeros((batch_size, self.hidden_size)).to(context.device)
        cell = torch.zeros((batch_size, self.hidden_size)).to(context.device)

        #initialize the input with embedding of the start token
        init_embed = self.embed(torch.tensor([self.vocab.wd_to_id[START_TOKEN]]).to(device)).reshape((1, self.embed_size))
        init_embed = torch.repeat_interleave(init_embed, batch_size, dim = 0).to(context.device)

        # info(f"Initial Embedding Shape: {init_embed.shape}")

        input = torch.cat([context, init_embed], dim = 1).to(context.device)

        #initialize the output_history and init_output
        outputs = []
        output = torch.zeros((batch_size, self.vocab_size)).to(context.device)
        
        
        for i in range(self.max_length):
            hidden, cell = self.lstm(input, (hidden, cell))
            output = self.linear(hidden)
            # output = self.softmax(output)
            outputs.append(output)
            
            #teacher forcing: 50% times
            r = torch.rand(1)
            if r>0.5:
                embedding = self.embed(target_tensor[:, i]).reshape((batch_size, self.embed_size)).to(context.device)
                input = torch.cat([context, embedding], dim = 1).to(context.device)              
            else:
                #add the embedding of the last prediction
                input = torch.cat([context, self.embed(torch.argmax(output, dim = 1))], dim = 1).to(context.device)
        # info(f"Outputs: {outputs}")
        return torch.stack(outputs).to(context.device), hidden, cell

### Utility Functions

In [6]:
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from tqdm import tqdm

plt.switch_backend('agg')
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
def saveModel(save_path, model_state, optimiser_state, loss):
    torch.save({
            'model_state_dict': model_state,
            'optimizer_state_dict':optimiser_state,
            'loss': loss,  
    }, save_path)
    

### Training Code.
- Dataloader automatically loads in batches. The data need not be modified by us.

In [7]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    total_loss = 0
    idx = 0
    for data in dataloader:
        idx+=1
        
        info(f"----Batch {idx}----")
        input_tensor, target_tensor = data['image'].to(device), data['formula'].to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        encoder_output = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_output, target_tensor)
        
        # print(encoder_output.device, 'My device')
        
        # print(f"Decoder OutDim : {decoder_outputs.shape}, Target Tensor Dim: {target_tensor.shape}")
        # print(f"Target tensor: {target_tensor[0][0]}")
        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()
        
        print(f'Loss for batch {idx} = {loss.item()}')

    return total_loss / len(dataloader)

def train(train_dataloader, encoder, decoder, n_epochs, save_interval = 2, learning_rate=0.001, print_every=1, plot_every=5):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss().to(device) #as stated in assignment
    
    # Print model's device
    # print("Encoder's device:", next(encoder.parameters()).device)

    pb = tqdm(range(1, n_epochs + 1))
    for epoch in pb:
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            
        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
        
        if epoch % save_interval == 0:
            saveModel(f'checkpoints/encoder_epoch_{epoch}.pt', encoder.state_dict(), encoder_optimizer.state_dict(), loss)
            saveModel(f'checkpoints/decoder_epoch_{epoch}.pt', decoder.state_dict(), decoder_optimizer.state_dict(), loss)
            

        pb.set_description('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg))
        
    showPlot(plot_losses)

## Training

In [8]:
batch_size = 32
vocab_size = 1000
CONTEXT_SIZE = 512
HIDDEN_SIZE = 512
EMBED_SIZE = 512
MAX_EXAMPLES = 1000
# image processing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x/255.0), #min-max normalisation
])

### Load Dataset and Dataloader

In [9]:
#part a
#train_csv_path = "/kaggle/input/converting-handwritten-equations-to-latex-code/col_774_A4_2023/SyntheticData/train.csv"
#image_root_path = "/kaggle/input/converting-handwritten-equations-to-latex-code/col_774_A4_2023/SyntheticData/images"
train_csv_path = "data/SyntheticData/train.csv"
image_root_path = "data/SyntheticData/images"
train_dataloader, train_dataset = get_dataloader(train_csv_path, image_root_path, batch_size, transform, max_examples=None)

Loading Dataset...
Loaded Dataset <bound method DataFrame.info of                 image                                            formula
0      74d337e8a0.png  $ \gamma _ { \Omega R , 5 } ^ { T } = - \gamma...
1      2d0f18f71d.png  $ l ^ { ( -- ) \underline { { m } } } u _ { \u...
2      6d9b9de88d.png  $ \left[ H , \gamma _ { i } ^ { \left( 2 \righ...
3      38c6d510bb.png  $ < a _ { i } > \; \propto \; \int _ { \omega ...
4      24537a86e3.png  $ \Psi ( \mu _ { 1 } , \ldots , \mu _ { K } ) ...
...               ...                                                ...
74995  1fa37e67d2.png  $ T _ { \theta } ^ { \theta } = - \frac { 1 } ...
74996  75518a26df.png  $ \alpha _ { + } = - 1 / \alpha _ { - } = \sqr...
74997  29f28cbc3a.png  $ d s ^ { 2 } = Z ^ { - 1 / 2 } \eta _ { \mu \...
74998  33ac7b385d.png  $ \tilde { H } _ { 0 } = \frac { 1 } { 2 } ( \...
74999  52672fbf76.png  $ \psi _ { \alpha \beta } = - g _ { \alpha \ga...

[75000 rows x 2 columns]>


### Create Model

In [10]:
#create a network instance
encoder = EncoderCNN().to(device)
decoder = Decoder(CONTEXT_SIZE, train_dataset.vocab, n_layers=1, hidden_size= HIDDEN_SIZE, embed_size=EMBED_SIZE,max_length=train_dataset.maxlen).to(device)
print(torch.cuda.get_device_name(0))

NVIDIA A100-PCIE-40GB


### Train

In [ ]:
train(train_dataloader, encoder, decoder, 10, save_interval = 2)

  0%|                                                                                             | 0/10 [00:00<?, ?it/s]

----Batch 1----
Loss for batch 1 = 6.336750030517578
----Batch 2----
Loss for batch 2 = 4.088656902313232
----Batch 3----
Loss for batch 3 = 1.8909673690795898
----Batch 4----
Loss for batch 4 = 0.934638261795044
----Batch 5----
Loss for batch 5 = 0.759469211101532
----Batch 6----
Loss for batch 6 = 0.8507074117660522
----Batch 7----
Loss for batch 7 = 0.7870851159095764
----Batch 8----
Loss for batch 8 = 0.7733004093170166
----Batch 9----
Loss for batch 9 = 0.7697874903678894
----Batch 10----
Loss for batch 10 = 0.7144590020179749
----Batch 11----
Loss for batch 11 = 0.7257136106491089
----Batch 12----
Loss for batch 12 = 0.7066599726676941
----Batch 13----
Loss for batch 13 = 0.7501876950263977
----Batch 14----
Loss for batch 14 = 0.7361814975738525
----Batch 15----
Loss for batch 15 = 0.6961299777030945
----Batch 16----
Loss for batch 16 = 0.7556687593460083
----Batch 17----
Loss for batch 17 = 0.6457914113998413
----Batch 18----
Loss for batch 18 = 0.7300599813461304
----Batch 19--

----Batch 147----
Loss for batch 147 = 0.6744541525840759
----Batch 148----
Loss for batch 148 = 0.6650529503822327
----Batch 149----
Loss for batch 149 = 0.6994333863258362
----Batch 150----
Loss for batch 150 = 0.6529591679573059
----Batch 151----
Loss for batch 151 = 0.6387475728988647
----Batch 152----
Loss for batch 152 = 0.6664289832115173
----Batch 153----
Loss for batch 153 = 0.7768464684486389
----Batch 154----
Loss for batch 154 = 0.7318217158317566
----Batch 155----
Loss for batch 155 = 0.7109777331352234
----Batch 156----
Loss for batch 156 = 0.7024853229522705
----Batch 157----
Loss for batch 157 = 0.6559823751449585
----Batch 158----
Loss for batch 158 = 0.6412087678909302
----Batch 159----
Loss for batch 159 = 0.8097293376922607
----Batch 160----
Loss for batch 160 = 0.7927542328834534
----Batch 161----
Loss for batch 161 = 0.7064275145530701
----Batch 162----
Loss for batch 162 = 0.7029843926429749
----Batch 163----
Loss for batch 163 = 0.6747865676879883
----Batch 164-

----Batch 289----
Loss for batch 289 = 0.7041324973106384
----Batch 290----
Loss for batch 290 = 0.7091871500015259
----Batch 291----
Loss for batch 291 = 0.775210976600647
----Batch 292----
Loss for batch 292 = 0.7401321530342102
----Batch 293----
Loss for batch 293 = 0.6901451945304871
----Batch 294----
Loss for batch 294 = 0.6835574507713318
----Batch 295----
Loss for batch 295 = 0.6795048117637634
----Batch 296----
Loss for batch 296 = 0.7966568470001221
----Batch 297----
Loss for batch 297 = 0.7475168108940125
----Batch 298----
Loss for batch 298 = 0.686545729637146
----Batch 299----
Loss for batch 299 = 0.6913046836853027
----Batch 300----
Loss for batch 300 = 0.6533486247062683
----Batch 301----
Loss for batch 301 = 0.6973468661308289
----Batch 302----
Loss for batch 302 = 0.7530636191368103
----Batch 303----
Loss for batch 303 = 0.6851115822792053
----Batch 304----
Loss for batch 304 = 0.7614176273345947
----Batch 305----
Loss for batch 305 = 0.7336505055427551
----Batch 306---

----Batch 431----
Loss for batch 431 = 0.6351765394210815
----Batch 432----
Loss for batch 432 = 0.7366238832473755
----Batch 433----
Loss for batch 433 = 0.7277689576148987
----Batch 434----
Loss for batch 434 = 0.8096972703933716
----Batch 435----
Loss for batch 435 = 0.8596280813217163
----Batch 436----
Loss for batch 436 = 0.6764630675315857
----Batch 437----
Loss for batch 437 = 0.788262665271759
----Batch 438----
Loss for batch 438 = 0.7149176597595215
----Batch 439----
Loss for batch 439 = 0.7677329778671265
----Batch 440----
Loss for batch 440 = 0.7264788150787354
----Batch 441----
Loss for batch 441 = 0.6461790800094604
----Batch 442----
Loss for batch 442 = 0.7471745014190674
----Batch 443----
Loss for batch 443 = 0.7360185980796814
----Batch 444----
Loss for batch 444 = 0.5982176661491394
----Batch 445----
Loss for batch 445 = 0.7458980679512024
----Batch 446----
Loss for batch 446 = 0.7365542054176331
----Batch 447----
Loss for batch 447 = 0.618071973323822
----Batch 448---

...

32m 48s (- 295m 17s) (1 10%) 0.7076:  10%|████▍                                       | 1/10 [32:48<4:55:17, 1968.61s/it]

Loss for batch 2344 = 0.7824202179908752
----Batch 1----
Loss for batch 1 = 0.6631912589073181
----Batch 2----
Loss for batch 2 = 0.6759697198867798
----Batch 3----
Loss for batch 3 = 0.7161123156547546
----Batch 4----
Loss for batch 4 = 0.7074294090270996
----Batch 5----
Loss for batch 5 = 0.7235667109489441
----Batch 6----
Loss for batch 6 = 0.7286194562911987
----Batch 7----
Loss for batch 7 = 0.6664547920227051
----Batch 8----
Loss for batch 8 = 0.6477235555648804
----Batch 9----
Loss for batch 9 = 0.7044108510017395
----Batch 10----
Loss for batch 10 = 0.7561968564987183
----Batch 11----
Loss for batch 11 = 0.7242017388343811
----Batch 12----
Loss for batch 12 = 0.6615690588951111
----Batch 13----
Loss for batch 13 = 0.7198619246482849
----Batch 14----
Loss for batch 14 = 0.7349472045898438
----Batch 15----
Loss for batch 15 = 0.7040112018585205
----Batch 16----
Loss for batch 16 = 0.7117294669151306
----Batch 17----
Loss for batch 17 = 0.6493587493896484
----Batch 18----
Loss for

Loss for batch 146 = 0.6538780927658081
----Batch 147----
Loss for batch 147 = 0.76242995262146
----Batch 148----
Loss for batch 148 = 0.6612457036972046
----Batch 149----
Loss for batch 149 = 0.662057638168335
----Batch 150----
Loss for batch 150 = 0.7472861409187317
----Batch 151----
Loss for batch 151 = 0.7550366520881653
----Batch 152----
Loss for batch 152 = 0.6840179562568665
----Batch 153----
Loss for batch 153 = 0.6201678514480591
----Batch 154----
Loss for batch 154 = 0.6488479375839233
----Batch 155----
Loss for batch 155 = 0.7210575342178345
----Batch 156----
Loss for batch 156 = 0.6416693925857544
----Batch 157----
Loss for batch 157 = 0.6642118096351624
----Batch 158----
Loss for batch 158 = 0.6829869747161865
----Batch 159----
Loss for batch 159 = 0.6531922817230225
----Batch 160----
Loss for batch 160 = 0.7419264316558838
----Batch 161----
Loss for batch 161 = 0.6603551506996155
----Batch 162----
Loss for batch 162 = 0.7270459532737732
----Batch 163----
Loss for batch 16

Loss for batch 288 = 0.7981297373771667
----Batch 289----
Loss for batch 289 = 0.7043896913528442
----Batch 290----
Loss for batch 290 = 0.7754504680633545
----Batch 291----
Loss for batch 291 = 0.7603468894958496
----Batch 292----
Loss for batch 292 = 0.7414373159408569
----Batch 293----
Loss for batch 293 = 0.7729094624519348
----Batch 294----
Loss for batch 294 = 0.709554135799408
----Batch 295----
Loss for batch 295 = 0.6676201224327087
----Batch 296----
Loss for batch 296 = 0.7248518466949463
----Batch 297----
Loss for batch 297 = 0.708402693271637
----Batch 298----
Loss for batch 298 = 0.693486750125885
----Batch 299----
Loss for batch 299 = 0.650813639163971
----Batch 300----
Loss for batch 300 = 0.751993715763092
----Batch 301----
Loss for batch 301 = 0.6769618988037109
----Batch 302----
Loss for batch 302 = 0.7759156823158264
----Batch 303----
Loss for batch 303 = 0.6885668635368347
----Batch 304----
Loss for batch 304 = 0.7498618364334106
----Batch 305----
Loss for batch 305 

Loss for batch 430 = 0.7392565608024597
----Batch 431----
Loss for batch 431 = 0.850979208946228
----Batch 432----
Loss for batch 432 = 0.6724632382392883
----Batch 433----
Loss for batch 433 = 0.7479549646377563
----Batch 434----
Loss for batch 434 = 0.6827015280723572
----Batch 435----
Loss for batch 435 = 0.7441122531890869
----Batch 436----
Loss for batch 436 = 0.6397472023963928
----Batch 437----
Loss for batch 437 = 0.7018383741378784
----Batch 438----
Loss for batch 438 = 0.6135950088500977
----Batch 439----
Loss for batch 439 = 0.6915257573127747
----Batch 440----
Loss for batch 440 = 0.7275986671447754
----Batch 441----
